In [ ]:
'''
变分自动编码器: MNIST图像生成
    只使用训练数据
    定义编码器和解码器，均由线性层构成
    编码器产生均值和方差进行重参数得到隐含变量，之后利用解码器进行重建
    使用二元交叉熵损失和KL散度作为最终损失
'''

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Create a directory if not exists
sample_dir = 'vae_samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [4]:
# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [5]:
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

In [6]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [7]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

/469], Reconst Loss: 10563.4980, KL Div: 3271.4248
Epoch[10/15], Step [60/469], Reconst Loss: 10674.3652, KL Div: 3283.5684
Epoch[10/15], Step [70/469], Reconst Loss: 9938.0234, KL Div: 3203.2808
Epoch[10/15], Step [80/469], Reconst Loss: 10683.5938, KL Div: 3359.8535
Epoch[10/15], Step [90/469], Reconst Loss: 10355.2607, KL Div: 3187.9998
Epoch[10/15], Step [100/469], Reconst Loss: 10113.1152, KL Div: 3170.6177
Epoch[10/15], Step [110/469], Reconst Loss: 9958.2070, KL Div: 3176.1445
Epoch[10/15], Step [120/469], Reconst Loss: 10205.7861, KL Div: 3155.8237
Epoch[10/15], Step [130/469], Reconst Loss: 10275.2510, KL Div: 3254.6748
Epoch[10/15], Step [140/469], Reconst Loss: 10442.8516, KL Div: 3196.7402
Epoch[10/15], Step [150/469], Reconst Loss: 10477.3164, KL Div: 3275.6045
Epoch[10/15], Step [160/469], Reconst Loss: 10687.8506, KL Div: 3218.0181
Epoch[10/15], Step [170/469], Reconst Loss: 10518.9844, KL Div: 3302.6875
Epoch[10/15], Step [180/469], Reconst Loss: 10230.9434, KL Div: 323